# Estimating Stellar Mass-to-Light Ratios using Colours

Based on the work by [Taylor et al. (2011)](https://academic.oup.com/mnras/article/418/3/1587/1060932) and maybe [Gallazzi & Bell (2009)](https://iopscience.iop.org/article/10.1088/0067-0049/185/2/253)


In [1]:
# May need to do this if running remote server with VS Code
%cd "/arc/home/IsaacCheng/coop_f2021/warmup"

/arc/home/IsaacCheng/coop_f2021/warmup


In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import astropy.units as u
import fits_plot_utils as utils
import dill

# Load and mask data


In [7]:
# # 
# # Load data
# # 
# vcc792_dist = 16.5 * u.Mpc
# # Signal
# zband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184.fits"
# iband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits"
# gband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
# uband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits"
# # Noise (sigma maps)
# znoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10184.fits"
# inoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184.fits"
# gnoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184.fits"
# unoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184.fits"
# # Mask (flap maps)
# zgood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.flag.3136_8588_6905_10184.fits"
# igood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.flag.3136_8588_6905_10184.fits"
# ggood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.flag.3136_8588_6905_10184.fits"
# ugood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.flag.3136_8588_6905_10184.fits"
# # 
# # Load NGVS data
# # 
# # Signal
# data_z, header_z = utils.load_img(zband_path)
# data_i, header_i = utils.load_img(iband_path)
# data_g, header_g = utils.load_img(gband_path)
# data_u, header_u = utils.load_img(uband_path)
# # Noise
# noise_z, header_noise_z = utils.load_img(znoise_path)
# noise_i, header_noise_i = utils.load_img(inoise_path)
# noise_g, header_noise_g = utils.load_img(gnoise_path)
# noise_u, header_noise_u = utils.load_img(unoise_path)
# # Masks
# good_z, header_good_z = utils.load_img(zgood_path, idx=1)
# good_i, header_good_i = utils.load_img(igood_path, idx=1)
# good_g, header_good_g = utils.load_img(ggood_path, idx=1)
# good_u, header_good_u = utils.load_img(ugood_path, idx=1)
# # 
# # VERTICO CO Data
# # 
# # co_path = "/arc/projects/vertico/products/release.v1.2/native/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_mom0_Msun.fits"
# co_path = "/arc/projects/vertico/products/release.v1.2/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_nyquistPix_9_arcsec_mom0_Msun.fits"
# data_co, header_co = utils.load_img(co_path)
# wcs_co = WCS(header_co)
# #
# # Mask the data and noise arrays
# #
# data_u_masked = np.ma.masked_array(data_u, mask=good_u == 0).filled(fill_value=np.nan)
# noise_u_masked = np.ma.masked_array(noise_u, mask=good_u == 0).filled(fill_value=np.nan)
# data_g_masked = np.ma.masked_array(data_g, mask=good_g == 0).filled(fill_value=np.nan)
# noise_g_masked = np.ma.masked_array(noise_g, mask=good_g == 0).filled(fill_value=np.nan)
# data_i_masked = np.ma.masked_array(data_i, mask=good_i == 0).filled(fill_value=np.nan)
# noise_i_masked = np.ma.masked_array(noise_i, mask=good_i == 0).filled(fill_value=np.nan)
# data_z_masked = np.ma.masked_array(data_z, mask=good_z == 0).filled(fill_value=np.nan)
# noise_z_masked = np.ma.masked_array(noise_z, mask=good_z == 0).filled(fill_value=np.nan)

In [31]:
# 
# Load binned data from pickle file
# 
g_infile = "/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184_BINNED_reproject_exact.pkl"
with open(g_infile, "rb") as f:
    g_file = dill.load(f)
    g_xs = g_file["x_coords"]
    g_ys = g_file["y_coords"]
    g_signal = g_file["signal_binned"]
    g_noise = g_file["noise_binned"]
    g_isgood = g_file["is_good"]
    g_wcs = g_file["wcs_binned"]
    g_wcs.array_shape = g_file["wcs_binned_array_shape"]
# 
i_infile = "/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184_BINNED_reproject_exact.pkl"
with open(i_infile, "rb") as f:
    i_file = dill.load(f)
    i_xs = i_file["x_coords"]
    i_ys = i_file["y_coords"]
    i_signal = i_file["signal_binned"]
    i_noise = i_file["noise_binned"]
    i_isgood = i_file["is_good"]
    i_wcs = i_file["wcs_binned"]
    i_wcs.array_shape = i_file["wcs_binned_array_shape"]
# 
# Load CO map data
# 
co_path = "/arc/projects/vertico/products/release.v1.2/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_nyquistPix_9_arcsec_mom0_Msun.fits"
data_co, header_co = utils.load_img(co_path)
wcs_co = WCS(header_co)

Filename: /arc/projects/vertico/products/release.v1.2/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_nyquistPix_9_arcsec_mom0_Msun.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      63   (42, 42)   float64   


In [10]:
gi_signal = -2.5 * np.log10(g_signal / i_signal)  # g- & i-values from binned data
# gi_noise = utils.calc_colour_err(g_signal, i_signal, g_noise, i_noise)
# 
# Calculate relative magnitude in i-band
# 
rel_mag_i = -2.5 * np.log10(i_signal) + 30.0  # AB magnitude system
# 
# Get absolute magnitudes in i-band
# 
vcc792_dist_pc = 16.5 * 1e6  # pc
abs_mag_i = rel_mag_i - 5 * (np.log10(vcc792_dist_pc) - 1)
abs_mag_i_sun = 4.58  # absolute AB magnitude of Sun in i-band
#
# Eq. (7) of Taylor et al. (2011)
#
log_MLi = -0.68 + 0.70 * gi_signal  # log10 of mass-to-light ratio in i-band
MLi = 10 ** log_MLi  # mass-to-light ratio in i-band
# 
# Get masses in solar units
# 
Li = 10 ** (-0.4 * (abs_mag_i - abs_mag_i_sun))  # luminosity in i-band
M = MLi * Li  # masses in solar units
# 
# Save to file
# 
Li_outfile = (
    "/arc/home/IsaacCheng/coop_f2021/binned_data/vcc792_MLi_BINNED_reproject_exact.txt"
)
id_arr = np.array(
    [str(x).zfill(2) + str(y).zfill(2) for x, y in zip(g_xs.flatten(), g_ys.flatten())]
)
Li_outdata = pd.DataFrame({"id": id_arr, "Li": Li.flatten(), "M": M.flatten()})
Li_outdata.to_csv(Li_outfile, sep=" ", index=False, header=True)
# 
# Calculating masses using Eq. (8) of Taylor et al. (2011)
# 
M2 = 10 ** (1.15 + 0.7 * gi_signal - 0.4 * abs_mag_i)
# print(M)
# print(M2)

In [45]:
%matplotlib widget
plt.close("all")
# 
# Plot mass map
# 
cmap = "viridis"
fig, ax = plt.subplots(subplot_kw={"projection": i_wcs})
img = ax.imshow(M, cmap=cmap, norm=LogNorm(vmin=2e6, vmax=1e8))
# img = ax.imshow(M, cmap=cmap, vmin=1e6, vmax=3e7)
# img = ax.imshow(M, cmap=cmap, vmin=1e6, vmax=5e7)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label(r"Stellar Mass Estimate ($M_*/M_\odot$)")
cbar.ax.yaxis.set_offset_position("left")
ax.contour(
    data_co,
    transform=ax.get_transform(WCS(header_co)),
    levels=range(0, 15, 5),
    colors="w"
)
ax.set_title("VCC 792 / NGC 4380: Stellar Mass Map Estimate")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
# fig.tight_layout()
# fig.savefig("imgs/vcc792_i-mass_estimate.pdf")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …